In [6]:
from mtcnn import MTCNN
from keras_facenet import FaceNet
import cv2
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize detector and embedder
detector = MTCNN()
embedder = FaceNet()

def l2_normalize(x):
    return x / np.linalg.norm(x)


In [7]:
# Folder containing known faces
photo_dir = "photos"

# Get embedding from image path
def get_face_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"[WARNING] Failed to load image: {img_path}")
        return None
    
    results = detector.detect_faces(img)
    if len(results) == 0:
        print(f"[INFO] No face detected in: {img_path}")
        return None
    
    face = results[0]
    x, y, w, h = face['box']
    x, y = max(0, x), max(0, y)
    
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (160, 160))
    
    embedding = embedder.embeddings([face_img])[0]
    embedding = l2_normalize(embedding)
    return embedding

# Build the face database from photo directory
def build_face_database(folder):
    database = {}
    for file in os.listdir(folder):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder, file)
            name = os.path.splitext(file)[0]
            print(f"Processing: {file}")
            embedding = get_face_embedding(path)
            if embedding is not None:
                database[name] = embedding
            else:
                print(f"[SKIPPED] {file}")
    return database

face_database = build_face_database(photo_dir)
print(f"✅ Loaded {len(face_database)} valid faces from Photo folder.")


Processing: 24WMA08802.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Processing: 24WMA08803.jpg
[INFO] No face detected in: photos\24WMA08803.jpg
[SKIPPED] 24WMA08803.jpg
Processing: 24WMH08807.jpg
[INFO] No face detected in: photos\24WMH08807.jpg
[SKIPPED] 24WMH08807.jpg
Processing: 24WMR08820.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Processing: 24WMR08821.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Processing: 24WMR08822.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Processing: 24WMR08824.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Processing: 24WMR08826.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Processing: 24WMR08827.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Processing: 24WMR08828.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Processing: 24WMR08829.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Processing: 24WMR08831.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Processing: 24WMR08832.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Processing: 24WMR08833.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Processing: 24WMR0

In [8]:
# Get top N matches with cosine similarity
def get_top_matches(face_img, database, top_n=3):
    face_img = cv2.resize(face_img, (160, 160))
    embedding = embedder.embeddings([face_img])[0]
    embedding = l2_normalize(embedding)

    similarities = []
    for name, db_emb in database.items():
        sim_score = cosine_similarity([embedding], [db_emb])[0][0]  # Higher is better
        similarities.append((name, sim_score))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Start webcam recognition
video = cv2.VideoCapture(0)
print("📷 Press 'q' to quit...")

threshold = 0.7  # Raised threshold

while True:
    ret, frame = video.read()
    if not ret:
        break

    results = detector.detect_faces(frame)
    
    for face in results:
        x, y, w, h = face['box']
        x, y = max(0, x), max(0, y)

        # Add margin to bounding box
        margin = 10
        x1 = max(0, x - margin)
        y1 = max(0, y - margin)
        x2 = min(frame.shape[1], x + w + margin)
        y2 = min(frame.shape[0], y + h + margin)

        face_img = frame[y1:y2, x1:x2]

        top_matches = get_top_matches(face_img, face_database)

        if top_matches and top_matches[0][1] > threshold:
            name = top_matches[0][0]
            similarity = top_matches[0][1]
            cv2.putText(frame, f"{name} ({similarity*100:.1f}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            print(f"\nTop 3 matches for face at ({x}, {y}):")
            for match_name, sim in top_matches:
                print(f"  {match_name}: {sim * 100:.2f}% similarity")
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            print(f"\n❌ No good match found for face at ({x}, {y}).")

    cv2.imshow('Face Recognition with MTCNN + FaceNet', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


📷 Press 'q' to quit...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step

Top 3 matches for face at (241, 227):
  24WMR08866: 77.22% similarity
  24WMR08863: 52.52% similarity
  24WMR08861: 51.75% similarity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step

Top 3 matches for face at (241, 223):
  24WMR08866: 73.73% similarity
  24WMR08832: 53.97% similarity
  24WMR08861: 51.00% similarity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step

Top 3 matches for face at (253, 220):
  24WMR08866: 80.22% similarity
  24WMR08863: 59.34% similarity
  24WMR08861: 57.33% similarity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

Top 3 matches for face at (252, 216):
  24WMR08866: 74.56% similarity
  24WMR08863: 57.97% similarity
  24WMR08861: 52.83% similarity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

Top 3 matches for face at (243, 223):
  24WMR08866: 73.41% similarity
  24WMR08863: 63.74% similarity
  24WMR08861: 51.53% similarity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

❌ No good match found for face at (251, 216).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [9]:
from mtcnn import MTCNN
from keras_facenet import FaceNet
import cv2
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize detector and embedder
detector = MTCNN()
embedder = FaceNet()

def l2_normalize(x):
    return x / np.linalg.norm(x)

# Folder containing known faces
photo_dir = "photos"

# Get embedding from image path
def get_face_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"[WARNING] Failed to load image: {img_path}")
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)

    if len(results) == 0:
        print(f"[INFO] No face detected in: {img_path}")
        return None

    face = results[0]
    x, y, w, h = face['box']
    x, y = max(0, x), max(0, y)

    face_img = img_rgb[y:y + h, x:x + w]
    face_img = cv2.resize(face_img, (160, 160))

    # Normalize pixel values to [-1, 1]
    face_img = face_img.astype('float32')
    face_img = (face_img - 127.5) / 128.0

    embedding = embedder.embeddings([face_img])[0]
    embedding = l2_normalize(embedding)
    return embedding

# Build the face database from photo directory
def build_face_database(folder):
    database = {}
    for file in os.listdir(folder):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder, file)
            name = os.path.splitext(file)[0]
            print(f"Processing: {file}")
            embedding = get_face_embedding(path)
            if embedding is not None:
                database[name] = embedding
            else:
                print(f"[SKIPPED] {file}")
    return database

face_database = build_face_database(photo_dir)
print(f"✅ Loaded {len(face_database)} valid faces from Photo folder.")

# Get top N matches with cosine similarity
def get_top_matches(face_img_bgr, database, top_n=3):
    face_img_rgb = cv2.cvtColor(face_img_bgr, cv2.COLOR_BGR2RGB)
    face_img_rgb = cv2.resize(face_img_rgb, (160, 160))
    face_img_rgb = face_img_rgb.astype('float32')
    face_img_rgb = (face_img_rgb - 127.5) / 128.0

    embedding = embedder.embeddings([face_img_rgb])[0]
    embedding = l2_normalize(embedding)

    similarities = []
    for name, db_emb in database.items():
        sim_score = cosine_similarity([embedding], [db_emb])[0][0]
        similarities.append((name, sim_score))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

# Start webcam recognition (only for the nearest/closest face)
video = cv2.VideoCapture(0)
print("📷 Press 'q' to quit...")

threshold = 0.7  # Similarity threshold

while True:
    ret, frame = video.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(frame_rgb)

    if results:
        # Sort faces by area (width * height), descending
        results.sort(key=lambda face: face['box'][2] * face['box'][3], reverse=True)

        # Process only the closest face (largest bounding box)
        face = results[0]
        x, y, w, h = face['box']
        x, y = max(0, x), max(0, y)

        # Add margin to bounding box
        margin = 10
        x1 = max(0, x - margin)
        y1 = max(0, y - margin)
        x2 = min(frame.shape[1], x + w + margin)
        y2 = min(frame.shape[0], y + h + margin)

        face_img = frame[y1:y2, x1:x2]

        top_matches = get_top_matches(face_img, face_database)

        if top_matches and top_matches[0][1] > threshold:
            name = top_matches[0][0]
            similarity = top_matches[0][1]
            cv2.putText(frame, f"{name} ({similarity*100:.1f}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            print(f"\nTop 3 matches for closest face at ({x}, {y}):")
            for match_name, sim in top_matches:
                print(f"  {match_name}: {sim * 100:.2f}% similarity")
        else:
            cv2.putText(frame, "Unknown", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            print(f"\n❌ No good match found for closest face at ({x}, {y}).")

    cv2.imshow('Face Recognition (Closest Face Only)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


Processing: 24WMA08802.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Processing: 24WMA08803.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Processing: 24WMH08807.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Processing: 24WMR08820.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Processing: 24WMR08821.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Processing: 24WMR08822.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Processing: 24WMR08824.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Processing: 24WMR08826.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Processing: 24WMR08827.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Processing: 24WMR08828.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Processing: 24WMR08829.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Processing: 24WMR08831.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Processing: 24WMR08832.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Processing: 24WMR08833.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Processing: 24WMR08834.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Processing: 24WMR08835.jpg
